# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas, string, re
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
import pickle

sw= stopwords.words('english')

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pandas.read_sql('InsertTableName', con=engine)
X = df["message"]
Y = df.drop(["message", "original","genre", 'id'], axis = 1)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    text =re.sub(r'[^a-zA-Z0-9]'," "  , text)
    text = text.lower()
    text = text.split()

    
    return [word for word in text if not word in sw]
    
def tokenize2(text):
    text= [char  if char not in  string.punctuation else " " for char in text.lower()]
    text="".join(text)
    #split text into individual words
    text = text.split()
    # Remove stop words  
    
    text  = [word for word in text if word not in sw]
    
    
    return text


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline(
    [ ("corpus",CountVectorizer(analyzer=tokenize)),
     ("tdif", TfidfTransformer()),
     ("classifier", MultiOutputClassifier( MultinomialNB()))
    ]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .25)
print(X_train.shape, Y_train.shape )


(19662,) (19662, 36)


In [10]:
pipeline.fit(X_train, Y_train)

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('corpus', CountVectorizer(analyzer=<function tokenize at 0x0940A348>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
 ...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def make_predictions(classifier, input_set_X, input_set_y):
    predictions  = classifier.predict(input_set_X)
    predictions = pandas.DataFrame(predictions, columns= input_set_y.columns)
    for y in input_set_y.columns:
        print("\n" +y + "  \n")
        print(classification_report(input_set_y[y],predictions[y]))
        print(confusion_matrix(input_set_y[y], predictions[y]))
        print("#" *20 )
    

In [12]:
predictions = pipeline.predict(X_test)

In [13]:
predictions.shape

(6554, 36)

In [14]:
predictions = pandas.DataFrame(predictions, columns= Y.columns)
predictions.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
Y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
11197,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16205,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6484,1,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
19387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
for y in Y.columns:
    print("\n" +y + "  \n")
    print(classification_report(Y_test[y],predictions[y]))
    print(confusion_matrix(Y_test[y], predictions[y]))
    print("#" *20 )
    #print("The f1 score for category {} is {}".format(y, f1_score(Y_test, predictions[y])) )
    


related  

              precision    recall  f1-score   support

           0       0.83      0.10      0.18      1570
           1       0.78      0.99      0.87      4984

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.80      0.55      0.53      6554
weighted avg       0.79      0.78      0.71      6554

[[ 159 1411]
 [  33 4951]]
####################

request  

              precision    recall  f1-score   support

           0       0.85      0.99      0.92      5388
           1       0.86      0.20      0.33      1166

   micro avg       0.85      0.85      0.85      6554
   macro avg       0.86      0.60      0.62      6554
weighted avg       0.85      0.85      0.81      6554

[[5351   37]
 [ 930  236]]
####################

offer  

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

   micro avg       1.00      1.00      1.00      655

c:\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



[[5994    0]
 [ 558    2]]
####################

medical_products  

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6234
           1       1.00      0.00      0.01       320

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.98      0.50      0.49      6554
weighted avg       0.95      0.95      0.93      6554

[[6234    0]
 [ 319    1]]
####################

search_and_rescue  

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6364
           1       0.00      0.00      0.00       190

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.94      0.97      0.96      6554

[[6364    0]
 [ 190    0]]
####################

security  

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6434
           1       0.00      0.00   

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6232
           1       0.00      0.00      0.00       322

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.90      0.95      0.93      6554

[[6232    0]
 [ 322    0]]
####################

weather_related  

              precision    recall  f1-score   support

           0       0.81      0.97      0.88      4718
           1       0.84      0.43      0.57      1836

   micro avg       0.82      0.82      0.82      6554
   macro avg       0.83      0.70      0.73      6554
weighted avg       0.82      0.82      0.80      6554

[[4564  154]
 [1044  792]]
####################

floods  

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6006
           1       0.67      0.00      0.01       548

   micro avg       0.92      0.92      0.92      6554
 

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {"classifier__estimator__alpha"  :[0.1, 0.2, 0.5,0.8, 1 ],
             # "classifier__estimator__fit_prior":[True, False],
              "tdif__norm" :["l1", "l2", None],
              "tdif__use_idf": [True, False],
              "corpus__analyzer":[tokenize, tokenize2 ]}

cv = GridSearchCV(pipeline, parameters, verbose=1, cv=6)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, Y_train)

Fitting 6 folds for each of 60 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.c

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

In [ ]:
make_predictions(cv, X_test, Y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
X_genre = df[["genre","message"]]
#no change in Y

X_genre_train, X_genre_test, Y_genre_train, Y_genre_test = train_test_split(X_genre,Y, test_size = .25)


In [ ]:
categorical_features  = ["genre"]
categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder())])

text_features = "message"
text_transformer =Pipeline(
        steps = [ ("corpus",CountVectorizer(analyzer=tokenize)),
           ("tfid", TfidfTransformer())
        ])


preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features),
        ('cat', categorical_transformer, categorical_features)])

end2end = Pipeline(steps = [("preprocessor",preprocessor),
                            ("classifier", MultiOutputClassifier(MultinomialNB()))])

                   
parameters = {
              "classifier__estimator__alpha"  :[0.1, 0.2, 0.5,0.8, 1 ],
              "preprocessor__text__corpus__analyzer": [tokenize, tokenize2],
              "preprocessor__text__tfid__norm": ["l1", "l2", None],
              "preprocessor__text__tfid__use_idf": [True, False] 
             }            

In [ ]:
cv_expanded = GridSearchCV(end2end, parameters, verbose=1, cv=6)
cv_expanded.fit(X_genre_train, Y_genre_train)

In [ ]:
make_predictions(cv_expanded, X_genre_test, Y_genre_test)

### 9. Export your model as a pickle file

In [ ]:
with open("message_model.pickle", "wb") as pickle_file:
    pickle.dump(cv,pickle_file)

In [ ]:
with open("message_genre_model.pickle", "wb") as pickle_file:
    pickle.dump(cv_expanded, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.